## 1. 手动遍历迭代器

问题：你想遍历一个可迭代对象中的所有元素，但是却不想使用for循环。

解决办法：为了手动的遍历可迭代对象，使用next（）函数并在代码中捕获StopIteration异常。

In [1]:
def manual_iter():
    with open('/etc/passwd') as f:
        try:
            while True:
                line = next(f)
                print(line,end='')
        except StopIteration:
            pass
        
manual_iter()

通常来讲， StopIteration 用来指示迭代的结尾。 然而，如果你手动使用上面演示的 next() 函数的话，你还可以通过返回一个指定值来标记结尾，比如 None 。 下面是示例：

In [2]:
with open('/etc/passwd') as f:
    while True:
        line = next(f,None)
        if line is None:
            break
        print(line,end='')

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:33:33

>> 大多数情况下，我们会使用for循环语句来遍历一个可迭代对象。但是，偶尔也需要对迭代做更加精确的控制，这时候了解底层迭代机制就显得尤为重要了。

In [3]:
# 下面的交互示例向我们演示了迭代期间所发生的基本细节：
items = [1,2,3]

In [4]:
# Get the iterator
it = iter(items)

In [5]:
# Run the iterator
next(it)

1

In [6]:
next(it)

2

In [7]:
next(it)

3

In [8]:
next(it)

StopIteration: 

## 2. 代理迭代

In [14]:
# 实际上你只需要定义一个__iter__()方法，将迭代操作代理到容器内部的对象中去。
class Node:
    def __init__(self,value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self,node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
# Example
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    # output Node(1), Node(2)
    for ch in root:
        print(ch)

Node(1)
Node(2)


>> 在上面代码中，` __iter__()` 方法只是简单的将迭代请求传递给内部的` _children` 属性。

Python的迭代器协议需要 __iter__() 方法返回一个实现了 __next__() 方法的迭代器对象。 如果你只是迭代遍历其他容器的内容，你无须担心底层是怎样实现的。你所要做的只是传递迭代请求既可。

这里的 iter() 函数的使用简化了代码， iter(s) 只是简单的通过调用 s.__iter__() 方法来返回对应的迭代器对象， 就跟 len(s) 会调用 s.__len__() 原理是一样的。



## 3. 使用生成器创建新的迭代模式

In [15]:
# 生成一个新的迭代器
def frange(start,stop,increment):
    x = start
    while x < stop:
        yield x
        x += increment

In [16]:
for n in frange(0,4,0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5


In [17]:
list(frange(0,1,0.125))

[0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875]

In [18]:
# 一个函数中需要有一个yield语句即可将其转换为一个生成器。跟普通函数不同的是，生成器
# 只能用来迭代操作。
def countdown(n):
    print('Starting to count from',n)
    while n > 0:
        yield n
        n -= 1
    print('Done!')

In [19]:
# create the generator, notice no output appears
c = countdown(3)
c

<generator object countdown at 0x10e6a4b10>

In [20]:
next(c)

Starting to count from 3


3

In [21]:
next(c)

2

In [22]:
next(c)

1

In [23]:
next(c)

Done!


StopIteration: 

>> 一个生成器函数主要特征是它只会回应在迭代中使用到的 next 操作。 一旦生成器函数返回退出，迭代终止。我们在迭代中通常使用的for语句会自动处理这些细节，所以你无需担心。

## 4. 实现迭代器协议

**最简单的方法使用一个生成器函数**

In [24]:
# 以深度优先方式i 遍历树形节点的生成器
class Node:
    def __init__(self,value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self,node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()
            
            
# Example 
if __name__ == "__main__":
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


>> 在这段代码中，depth_first()方法简单直观。它首先返回自己本身并迭代每一个子节点通过调用子节点的depth_first()方法使用yield_from语句返回对应的元素

Python的迭代器协议要求一个__iter__()方法返回一个特殊的迭代器对象，这个迭代器对象实现了__next__()方法并通过StopIteration异常标识迭代的完成。但是，实现这个通常会比较繁琐。下面这种方式重新关联迭代器类实现depth_first()方法：

In [25]:
class Node2:
    def __init__(self,value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self,node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        return DepthFirstIterator(self)
    
    
class DepthFirstIterator(object):
    '''
    Depth-first traversal
    '''
    
    def __init__(self,start_node):
        self._node = start_node
        self._chirldren_iter = None
        self._child_iter = None
        
        def __iter__(self):
            return self
        
        def __next__(self):
            # return myself if just started; create an iterator for children
            if self._children_iter is None:
                self._children_iter = iter(self._node)
                return self._node
            
            elif self._child_iter:
                try:
                    nextchild = next(self._child_iter)
                    return nextchild
                except StopIteration:
                    self._child_iter = None
                    return next(self)
            else:
                self._child_iter = next(self._children_iter).depth_first()
                return next(self)

## 5. 反向迭代

In [26]:
# 使用内置的reversed()函数即可

In [27]:
a = [1,2,3,4]

In [28]:
for x in reversed(a):
    print(x)

4
3
2
1


>> 反向迭代仅仅当对象的大小可预先确定或者实现了__reversed__()的特殊方法时才能生效。如果两者都不符合，那你必须先将对象转换成一个列表才行

In [29]:
# 很多程序员并不知道可以通过在自定义类上实现__reversed__()方法来实现反向迭代
class Countdown:
    def __init__(self,start):
        self.start = start
    
    # forward iterator
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1 
            
    # reverse iterator
    def __reversed__(self):
        n = 1 
        while n <= self.start:
            yield n
            n += 1
        
        
for rr in reversed(Countdown(30)):
    print(rr)
    

print('*'* 50)
for rr in Countdown(30):
    print(rr)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
**************************************************
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


## 6. 带有外部状态的生成器函数

如果你想让你的生成器暴露外部状态给用户，别忘了你可以简单的将它实现为一个类，然后把生成器函数放到__iter__()方法中去。

In [30]:
from collections import deque

class linehistory:
    def __init__(self,lines,histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for lineno,line in enumerate(self.lines,1):
            self.history.append(lineno,line)
            yield line
            
    def clear(self):
        self.history.clear()

>> 关于生成器，很容易掉进函数无所不能的陷阱。如果生成器函数需要跟你的程序其他部分打交道的话，可能会导致你的代码异常的复杂。如果这种情况的话，可以考虑使用上面介绍的定义类的方式。在`__iter__()`方法中定义你的生成器不会改变你任何的算法逻辑。

## 7. 迭代器切片

函数itertools.islice()正好适用于在迭代器和生成器上做切片操作。

In [31]:
def count(n):
    while True:
        yield n
        n += 1

In [32]:
c = count(0)

In [33]:
c[10:20]

TypeError: 'generator' object is not subscriptable

In [34]:
# Now using islice()]
import itertools


for x in itertools.islice(c,10,20):
    print(x)

10
11
12
13
14
15
16
17
18
19


>> 迭代器和生成器不能使用标准的切片操作。因为他们的长度事先我们并不知道（并且也没有实现索引）。函数islice()返回一个可以生成指定元素的迭代器，它通过遍历并丢弃知道切片开始索引位置的所有元素。然后才开始一个个的返回元素，并直到切片结束索引位置。

>> 这里要着重强调的一点是islice()会消耗掉传入的迭代器中的数据。必须考虑到迭代器是不可逆的这个事实。所以如果你需要之后再次访问这个迭代器的话，那你就得先将它里面的数据放入一个列表中

## 8. 跳过可迭代对象的开始部分

In [35]:
# itertools模块中有一些函数可以完成这个任务

In [36]:
with open('/etc/passwd') as f:
    for line in f:
        print(line,end=' ')

##
 # User Database
 # 
 # Note that this file is consulted directly only when the system is running
 # in single-user mode.  At other times this information is provided by
 # Open Directory.
 #
 # See the opendirectoryd(8) man page for additional information about
 # Open Directory.
 ##
 nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
 root:*:0:0:System Administrator:/var/root:/bin/sh
 daemon:*:1:1:System Services:/var/root:/usr/bin/false
 _uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
 _taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
 _networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
 _installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
 _lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
 _postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
 _scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
 _ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/fal

In [37]:
# 如果你想跳过开始部分的注释的话，可以这样做：
from itertools import dropwhile

with open('/etc/passwd') as f:
    for line in dropwhile(lambda line:line.startswith('#'),f):
        print(line,end='')

nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:33:33:Mac App Store Service:/var/empty:/usr/bin/false
_mcxalr:*:54:54:MCX AppLaunch:/var/empty:/usr/bin/false
_appleevents:*:55:55:AppleEvents Daemon:/var/empty:/usr/bin/false
_geod:*:56:56:Geo Services Daemon:/var/db/geod:/usr/bin/false
_devdocs:*:59:59:Developer Documentation:/var/e

这个例子是基于根据某个测试数据跳过开始的元素。如果你已经知道了要跳过元素的个数，那么可以使用itertools.islice()来代替：

In [38]:
from itertools import islice

items = ['a','b','c',1,4,10,15]
for x in islice(items,3,None):
    print(x)

1
4
10
15


>> 在这个例子中，islice()函数最后那个None参数指定了你要获取从第3个到最后的所有元素，如果None和3的位置对调，意思是仅仅获取前三个元素（这个跟切片的相反操作[3:]和[:3]）。

## 9. 排列组合的迭代

In [39]:
# itertools模块提供了三个函数来解决这类问题，其中一个是itertools.permutation(),它
# 接受一个集合并产生一个元素序列，每个元组由集合中国呢所有元素的一个可能排列组成。
items = ['a','b','c']

from itertools import permutations

for p in permutations(items):
    print(p)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


In [40]:
# 如果你想得到指定长度的所有排列，可以传递一个可选的长度参数
for p in permutations(items,2):
    print(p)

('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')


In [41]:
# 使用itertools.combinations()可得到输入集合中元素的所有的组合
from itertools import combinations

for c in combinations(items,2):
    print(c)

('a', 'b')
('a', 'c')
('b', 'c')


In [42]:
for c in combinations(items,3):
    print(c)

('a', 'b', 'c')


In [43]:
for c in combinations(items,1):
    print(c)

('a',)
('b',)
('c',)


>> 对于combinations()来说，元素的顺序已经不重要了。也就是说，组合('a','b')和组合('b','a')其实是一样的。

In [44]:
# 函数itertools.combinations_with_replacement()允许同一个元素被多次选择
from itertools import combinations_with_replacement

for c in combinations_with_replacement(items,3):
    print(c)

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'c')
('a', 'b', 'b')
('a', 'b', 'c')
('a', 'c', 'c')
('b', 'b', 'b')
('b', 'b', 'c')
('b', 'c', 'c')
('c', 'c', 'c')


>> 这一小节我们向你展示的仅仅是 itertools 模块的一部分功能。 尽管你也可以自己手动实现排列组合算法，但是这样做得要花点脑力。 当我们碰到看上去有些复杂的迭代问题时，最好可以先去看看itertools模块。 如果这个问题很普遍，那么很有可能会在里面找到解决方案！

## 10. 序列上索引值迭代

In [45]:
# 内置的enumerate()函数可以很好的解决这个问题
my_list = ['a','b','c']
for idx,val in enumerate(my_list):
    print(idx,val)

0 a
1 b
2 c


In [46]:
# 为了按传统行号输出（行号从1开始），你可以传递一个开始参数
my_list =['a','b','c']
for idx,val in enumerate(my_list,1):
    print(idx,val)

1 a
2 b
3 c


In [47]:
# 这个情况在你遍历文件时想在错误消息中使用行号定位时候非常有用
def parse_date(filename):
    with open(filenname,'rt') as f:
        for lineno,line in enumerate(f,1):
            fields = line.split()
            try:
                count = int(fields[1])
            except ValueError as r:
                print('Line {}: Parse error: {}'.format(lineno,e))

>> enumerate()对于跟踪某些值在列表中出现的位置是很有用的。所以，如果你想将一个文件中国出现的单词映射到它出现的行号上去，可以很容易的利用enumerate()来完成

In [48]:
from collections import defaultdict

word_summary = defaultdict(list)

with open('somefile.txt','r') as f:
    lines = f.readlines()
    
for idx,lines in enumerate(lines):
    # create a list of words in current line
    words = [w.strip().lower() for w in line.split()]
    for word in words:
        word_summary[word].append(idx)

In [49]:
word_summary

defaultdict(list,
            {'_reportmemoryexception:*:269:269:reportmemoryexception:/var/db/reportmemoryexception:/usr/bin/false': [0,
              1,
              2,
              3,
              4,
              5,
              6,
              7,
              8,
              9,
              10,
              11,
              12,
              13,
              14,
              15,
              16,
              17,
              18,
              19,
              20,
              21,
              22,
              23,
              24,
              25,
              26,
              27,
              28,
              29,
              30,
              31,
              32,
              33,
              34,
              35,
              36,
              37,
              38,
              39,
              40,
              41,
              42,
              43,
              44,
              45,
              46,
              47,
              48,
       

>> 如果你处理完文件后打印 word_summary ，会发现它是一个字典(准确来讲是一个 defaultdict )， 对于每个单词有一个 key ，每个 key 对应的值是一个由这个单词出现的行号组成的列表。 如果某个单词在一行中出现过两次，那么这个行号也会出现两次， 同时也可以作为文本的一个简单统计。

## 11. 同时迭代多个序列

In [50]:
# 为了同时迭代多个序列，使用zip()函数
xpts = [1,5,4,2,10,7]
ypts = [101,78,37,15,62,99]
for x,y in zip(xpts,ypts):
    print(x,y)

1 101
5 78
4 37
2 15
10 62
7 99


>> zip(a,b)会生成一个可返回元组（x,y)的迭代器，其中x来自a，y来自b。一旦其中某个序列到底结尾了，迭代宣告结束。因此迭代长度跟阐述中最短序列长度一致。

In [51]:
a = [1,2,3]
b = ['w','x','y','z']
for i in zip(a,b):
    print(i)

(1, 'w')
(2, 'x')
(3, 'y')


In [52]:
# 如果这个不是你想要的效果，那么还可以使用itertools.zip_longest()函数来代替
from itertools import zip_longest

for i in zip_longest(a,b,fillvalue=0):
    print(i)

(1, 'w')
(2, 'x')
(3, 'y')
(0, 'z')


当你想成对处理数据的时候zip()函数是很有用的。比如，假设你头列表和一个值列表，就像下面这样：

In [53]:
headers = ['name','shares','price']
values = ['ACME',100,490.1]

In [54]:
s = dict(zip(headers,values))
s

{'name': 'ACME', 'shares': 100, 'price': 490.1}

In [55]:
# 或者你也可以想下面这样产生输出
for name,value in zip(headers,values):
    print(name,'=',value)

name = ACME
shares = 100
price = 490.1


In [56]:
# 虽然不常见，但是zip()可以接受多余两个的序列的参数。这时候所生成的结果元组中元素个数
# 跟输入序列个数一样
a = [1,2,3]
b = [10,11,12]
c = ['x','y','z']
for i in zip(a,b,c):
    print(i)

(1, 10, 'x')
(2, 11, 'y')
(3, 12, 'z')


In [57]:
# 最后强调一点的是，zip()会创建一个迭代器来作为结果返回。如果你需要将结对的值存储在列
# 表中，要使用list（）函数。
zip(a,b)

In [58]:
list(zip(a,b))

[(1, 10), (2, 11), (3, 12)]

## 12. 不同集合上元素的迭代

In [59]:
# itertools.chain()方法可以用来简化这个任务。它接受一个可迭代对象列表作为输入，
# 并返回一个迭代器，有效的屏蔽掉在多个容器中迭代的细节。

In [60]:
from itertools import chain

a = [1,2,3,4]
b = ['x','y','z']

for x in chain(a,b):
    print(x)

1
2
3
4
x
y
z


In [61]:
# 使用chain()的一个常见场景是当你想对不同集合中所有元素执行某些操作的时候。比如：
# Various working sets of items
active_items = set()
inactive_items = set()

# Iterate over all items
for item in chain(active_items,inactive_items):
    # Process item
    pass

In [62]:
# 这种解决方案要比下面这样这样使用两个单独的循环更加优雅
for item in active_items:
    # Process items
    pass

for item in inactive_items:
    # Process items
    pass

## 13. 创建数据处理管道

生成器函数是一个实现管道机制的好办法。为了演示，假设你要处理一个非常大的日志数据目录：

```
foo/
    access-log-012007.gz
    access-log-022007.gz
    access-log-032007.gz
    ...
    access-log-012008
bar/
    access-log-092007.bz2
    ...
    access-log-022008


```
每个日志文件包含这样的数据：
```
124.115.6.12 - - [10/Jul/2012:00:18:50 -0500] "GET /robots.txt ..." 200 71
210.212.209.67 - - [10/Jul/2012:00:18:51 -0500] "GET /ply/ ..." 200 11875
210.212.209.67 - - [10/Jul/2012:00:18:51 -0500] "GET /favicon.ico ..." 404 369
61.135.216.105 - - [10/Jul/2012:00:20:04 -0500] "GET /blog/atom.xml ..." 304 -
```




In [63]:
## 为了处理这些文件，你可以定义一个有多个执行特定任务独立任务的简单生成器函数组成的容器
# 就行这样的

import os
import fnmatch
import gzip
import bz2
import re

def gen_find(filepat,top):
    '''
    Find all filename in a directionary tree that match a  shell wildcard pattern
    '''
    for path,dirlist,filelist in os.walk(top):
        for name in fnmatch.filter(filelist,filepat):
            yield os.path.join(path,name)
            
def gen_opener(filenames):
    '''
    Open a sequence of filenames oen at a time producing a file object.
    The file is closed immediately when proceeding to the next iteration
    
    '''
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename,'rt')
        elif filename.endswith('.bz2'):
            f = bz.open(filename,'rt')
        else:
            f = open(filename,'rt')
            
        yield f
        f.close()
        
def gen_concatenate(iterators):
    '''
    
    Chain a sequence of iterators together into a single sequence
    
    '''
    for it in iterators:
        yield from it
        
def gen_grep(pattern,lines):
    '''
    Look for a regex pattern in a sequence of lines
    '''
    pat = re.compile(pattern)
    for line in lines:
        if pat.search(line):
            yield line

为了理解上述代码，重点是要明白 yield 语句作为数据的生产者而 for 循环语句作为数据的消费者。 当这些生成器被连在一起后，每个 yield 会将一个单独的数据元素传递给迭代处理管道的下一阶段。 在例子最后部分， sum() 函数是最终的程序驱动者，每次从生成器管道中提取出一个元素。

这种方式一个非常好的特点是每个生成器函数很小并且都是独立的。这样的话就很容易编写和维护它们了。 很多时候，这些函数如果比较通用的话可以在其他场景重复使用。 并且最终将这些组件组合起来的代码看上去非常简单，也很容易理解。

使用这种方式的内存效率也不得不提。上述代码即便是在一个超大型文件目录中也能工作的很好。 事实上，由于使用了迭代方式处理，代码运行过程中只需要很小很小的内存。

在调用 gen_concatenate() 函数的时候你可能会有些不太明白。 这个函数的目的是将输入序列拼接成一个很长的行序列。 itertools.chain() 函数同样有类似的功能，但是它需要将所有可迭代对象最为参数传入。 在上面这个例子中，你可能会写类似这样的语句 lines = itertools.chain(*files) ， 这将导致 gen_opener() 生成器被提前全部消费掉。 但由于 gen_opener() 生成器每次生成一个打开过的文件， 等到下一个迭代步骤时文件就关闭了，因此 chain() 在这里不能这样使用。 上面的方案可以避免这种情况。

gen_concatenate() 函数中出现过 yield from 语句，它将 yield 操作代理到父生成器上去。 语句 yield from it 简单的返回生成器 it 所产生的所有值。 关于这个我们在4.14小节会有更进一步的描述。

最后还有一点需要注意的是，管道方式并不是万能的。 有时候你想立即处理所有数据。 然而，即便是这种情况，使用生成器管道也可以将这类问题从逻辑上变为工作流的处理方式。

David Beazley 在他的 Generator Tricks for Systems Programmers 教程中对于这种技术有非常深入的讲解。可以参考这个教程获取更多的信息。

## 14. 展开嵌套的序列

In [64]:
from collections import Iterable

def flatten(items,ignore_type=(str,bytes)):
    for x in items:
        if isinstance(x,Iterable) and not isinstance(x,ignore_type):
            yield from flatten(x)
        else:
            yield x

In [65]:
items = [1,2,[3,4,[5,6],7],8]

In [66]:
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8


>> 在上面代码中， isinstance(x, Iterable) 检查某个元素是否是可迭代的。 如果是的话， yield from 就会返回所有子例程的值。最终返回结果就是一个没有嵌套的简单序列了。

In [67]:
items = ['Dave','Paula',['Thomas','Lewis']]

In [68]:
for x in flatten(items):
    print(x)

Dave
Paula
Thomas
Lewis


## 15. 顺序迭代合并的排序迭代对象

In [69]:
# heapq.merge()函数可以帮你解决这个问题
import heapq

a = [1,4,7,10]
b = [2,5,6,11]

for c in heapq.merge(a,b):
    print(c)

1
2
4
5
6
7
10
11


heapq.merge()可迭代特性意味着它不会迪马读取所有序列。这就意味着你可以在非常长的序列中使用它，而不会有太大的开销。

```
with open('sorted_file','rt') as file1,\
    open('sorted_file','rt') as file2,\
    open('merged_file','wt') as outf:
    
    for line in heapq.merge(file1,file2):
        outf.write(line)
        
```

>> 有一点要强调的是 heapq.merge() 需要所有输入序列必须是排过序的。 特别的，它并不会预先读取所有数据到堆栈中或者预先排序，也不会对输入做任何的排序检测。 它仅仅是检查所有序列的开始部分并返回最小的那个，这个过程一直会持续直到所有输入序列中的元素都被遍历完。

## 16. 迭代器代替while无限循环

In [70]:
# 一个常见的IO操作程序可能会想下面这样：
CHUNKSIZE = 8192

def reader(s):
    while True:
        data = s.recv(CHUNKSIZE)
        if data == b'':
            break
        process_data(data)

In [71]:
# 这种代码通常可以使用iter()代替，如下所示：
def reader2(s):
    for chunk in iter(lambda: s.recv(CHUNKSIZE),b''):
        pass
    #process_data(data)

In [72]:
import sys

f = open('/etc/passwd')

for chunk in iter(lambda: f.read(10),''):
    n = sys.stdout.write(chunk)

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.
##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false
_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:33:33

>> iter 函数一个鲜为人知的特性是它接受一个可选的 callable 对象和一个标记(结尾)值作为输入参数。 当以这种方式使用的时候，它会创建一个迭代器， 这个迭代器会不断调用 callable 对象直到返回值和标记值相等为止。